# Blue Brain BioExplorer
![](../bioexplorer_banner.png)

In [ ]:
from bioexplorer import BioExplorer, Protein, Surfactant, Vector3, Quaternion, MovieMaker
import nglview

be = BioExplorer('localhost:5000')

In [ ]:
status = be.reset()

### Resources

In [ ]:
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'

### Configuration

In [ ]:
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS_AND_STICKS
protein_load_hydrogen = False

## Camera

In [ ]:
status = be.core_api().set_camera(
    orientation=[0.820, -0.219, -0.177, 0.496],
    position=[-36.545, -85.005, -48.031],
    target=[19.865, -3.574, 1.045])

## Scene description

#### Surfactant A

In [ ]:
nglview.show_file(surfactant_head_source)

In [ ]:
nglview.show_file(surfactant_branch_source)

In [ ]:
def add_surfactant_a(name, position, rotation, random_seed):
    surfactant_a = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_A, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_a, 
        representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=position, rotation=rotation,
        random_seed=random_seed)

status = add_surfactant_a(
    name='Surfactant-A 1', random_seed=2,
    position=Vector3(), rotation=Quaternion(0.707, 0.707, 0, 0))

## Materials

In [ ]:
be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)

for model in be.core_api().scene.models:
    model_id = model['id']
    model_name = model['name']
    if be.NAME_COLLAGEN in model_name:
        material_ids = list(be.get_material_ids(model_id)['ids'])
        nb_materials = len(material_ids)
        print(model_name)
        palette = list()
        emissions = list()
        for i in range(nb_materials):
            palette.append([1,1,1])
            emissions.append(0.2)
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids,
            diffuse_colors=palette, specular_colors=palette,
            emissions=emissions
        )
status = be.core_api().scene.commit()

## Rendering settings

In [ ]:
status = be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 500
params.shadows = 1.0
params.soft_shadows = 0.3
params.fog_start = 1500
params.fog_thickness = 1500
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

## Snapshot

In [ ]:
mm = MovieMaker(be)

In [ ]:
mm.create_snapshot(
    path='/tmp', base_name='surfactant-a',
    samples_per_pixel=64, size=[512, 512])